In [26]:
female_ids = [2662, 4425, 14495, 15367, 15368, 15369, 17770, 17775, 18120, 18121, 
              18181, 18384, 18473, 18475, 2, 23164, 23167, 23170, 23179, 23892, 
              23899, 23901, 24267, 24269, 24486, 24490, 24494, 24496, 24498, 24985, 
              25023, 25192, 28348, 30858, 30859, 33927, 35461, 40590, 40728, 42619, 
              42620, 42627, 42730, 42808, 43390, 43394, 4426, 4429, 4435, 4472, 4524, 
              613, 6870, 6871, 6873, 6874, 6904, 6982, 6983, 7141, 7144, 8135, 8136, 
              8144, 8301, 8302, 8498, 8500, 9]

male_ids = [1, 10681, 10682, 14494, 14784, 14785, 15365, 15366, 17768, 17769, 17893, 
            18385, 18472, 18476, 18478, 18565, 18566, 23177, 23178, 23903, 24487, 
            24725, 24937, 24938, 24979, 24982, 24984, 25024, 25190, 28353, 28357, 
            28358, 28546, 33925, 3538, 4, 40587, 40588, 40727, 40729, 40730, 42618, 
            42622, 42623, 42625, 42626, 42729, 42809, 42961, 43389, 43391, 43392, 
            43393, 4434, 4436, 4437, 4470, 4471, 4473, 5, 6, 6872, 6875, 6902, 6903, 
            6905, 7143, 8146, 8298, 8425, 43396, 43397, 43398, 43400, 43401]

In [2]:
import requests
import time
from pathlib import Path
import hashlib
from IPython.display import clear_output, display, HTML
import PIL.Image
from io import BytesIO
import cv2
import numpy as np

class GenderDetector:
    def __init__(self):
        """Initialize gender detector with pre-trained models"""
        print("Initializing gender detection model...")
        
        # Model URLs
        model_url = "https://drive.google.com/uc?id=1W_moLzMlGiELyPxWiYQJ9KFaXroQ_NFQ"
        config_url = "https://drive.google.com/uc?id=1AW3WduLk1haTVAxHOkVS_BEzel1WXQHP"
        
        # File paths
        self.model_path = "gender_net.caffemodel"
        self.config_path = "gender_deploy.prototxt"
        
        # Download files if they don't exist
        if not Path(self.model_path).exists():
            print("Downloading model file...")
            self._download_file(model_url, self.model_path)
            
        if not Path(self.config_path).exists():
            print("Downloading config file...")
            self._download_file(config_url, self.config_path)
        
        # Load model
        self.net = cv2.dnn.readNet(self.model_path, self.config_path)
        
        if self.net.empty():
            raise ValueError("Failed to load the model")
            
        self.gender_list = ['Male', 'Female']

    def _download_file(self, url, filename):
        """Download a file from URL"""
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            
            with open(filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"Successfully downloaded {filename}")
        except Exception as e:
            print(f"Error downloading {filename}: {e}")
            raise

    def detect_gender(self, image):
        """Detect gender in the given image"""
        # Resize image to expected dimensions
        h, w = image.shape[:2]
        if h != 227 or w != 227:
            image = cv2.resize(image, (227, 227))
            
        # Pre-process image
        blob = cv2.dnn.blobFromImage(
            image, 
            1.0,
            (227, 227),
            (78.4263377603, 87.7689143744, 114.895847746),
            swapRB=False
        )
        
        # Forward pass
        self.net.setInput(blob)
        gender_preds = self.net.forward()
        
        # Get prediction
        gender = self.gender_list[gender_preds[0].argmax()]
        confidence = float(gender_preds[0].max())
        
        return gender, confidence

def download_faces(female_ids, male_ids, output_dir='generated_faces', delay=1.0):
    """Download AI-generated faces using specific ID lists"""
    # Convert ID lists to sets for easier removal
    female_ids = set(map(str, female_ids))
    male_ids = set(map(str, male_ids))
    
    # Initialize gender detector
    detector = GenderDetector()
    
    # Create output directories
    output_path = Path(output_dir)
    male_path = output_path / 'male'
    female_path = output_path / 'female'
    
    for path in [male_path, female_path]:
        path.mkdir(parents=True, exist_ok=True)
    
    # Track statistics
    stats = {'Male': 0, 'Female': 0}
    original_counts = {'Male': len(male_ids), 'Female': len(female_ids)}
    seen_hashes = set()
    
    url = 'https://thispersondoesnotexist.com'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    while female_ids or male_ids:
        try:
            # Download image
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            
            # Get image data and calculate hash
            img_data = response.content
            img_hash = hashlib.md5(img_data).hexdigest()
            
            # Skip if duplicate
            if img_hash in seen_hashes:
                continue
            
            # Convert to OpenCV format for gender detection
            nparr = np.frombuffer(img_data, np.uint8)
            img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            
            if img is None:
                print("Failed to decode image")
                continue
                
            # Log original image dimensions
            h, w = img.shape[:2]
            print(f"Original image dimensions: {w}x{h}")
            
            # Detect gender
            try:
                gender, confidence = detector.detect_gender(img)
            except Exception as e:
                print(f"Error in gender detection: {e}")
                continue
            
            # Skip if confidence is low or if we don't need this gender anymore
            if confidence < 0.7:
                print(f"Low confidence detection: {confidence:.2f}")
                continue
                
            if (gender == 'Male' and not male_ids) or (gender == 'Female' and not female_ids):
                print(f"Skipping {gender}, already have enough")
                continue
                
            # Get next ID from appropriate list
            if gender == 'Male':
                current_id = male_ids.pop()
                save_dir = male_path
            else:
                current_id = female_ids.pop()
                save_dir = female_path
            
            # Save image
            image_path = save_dir / f"{current_id}.jpg"
            with open(image_path, 'wb') as f:
                f.write(img_data)
            
            seen_hashes.add(img_hash)
            stats[gender] += 1
            
            # Display progress
            clear_output(wait=True)
            print(f"Progress:")
            print(f"Males: {stats['Male']}/{original_counts['Male']} ({len(male_ids)} IDs remaining)")
            print(f"Females: {stats['Female']}/{original_counts['Female']} ({len(female_ids)} IDs remaining)")
            print(f"Last detection confidence: {confidence:.2f}")
            
            # Display progress bars
            male_progress = f"""
            Males: <progress value="{stats['Male']}" max="{original_counts['Male']}" 
                style="width: 100%; height: 20px;">
            </progress>
            """
            female_progress = f"""
            Females: <progress value="{stats['Female']}" max="{original_counts['Female']}" 
                style="width: 100%; height: 20px;">
            </progress>
            """
            display(HTML(male_progress + female_progress))
            
            time.sleep(delay)
            
        except Exception as e:
            print(f"\nError: {e}")
            time.sleep(delay * 2)
            continue
            
    print(f"\nCompleted! All IDs have been assigned:")
    print(f"Males: {stats['Male']}/{original_counts['Male']}")
    print(f"Females: {stats['Female']}/{original_counts['Female']}")
    print(f"Images saved in: {output_path.absolute()}")
    return output_path, stats

# Define ID lists
female_ids = [2662, 4425, 14495, 15367, 15368, 15369, 17770, 17775, 18120, 18121, 
              18181, 18384, 18473, 18475, 2, 23164, 23167, 23170, 23179, 23892, 
              23899, 23901, 24267, 24269, 24486, 24490, 24494, 24496, 24498, 24985, 
              25023, 25192, 28348, 30858, 30859, 33927, 35461, 40590, 40728, 42619, 
              42620, 42627, 42730, 42808, 43390, 43394, 4426, 4429, 4435, 4472, 4524, 
              613, 6870, 6871, 6873, 6874, 6904, 6982, 6983, 7141, 7144, 8135, 8136, 
              8144, 8301, 8302, 8498, 8500, 9]

male_ids = [1, 10681, 10682, 14494, 14784, 14785, 15365, 15366, 17768, 17769, 17893, 
            18385, 18472, 18476, 18478, 18565, 18566, 23177, 23178, 23903, 24487, 
            24725, 24937, 24938, 24979, 24982, 24984, 25024, 25190, 28353, 28357, 
            28358, 28546, 33925, 3538, 4, 40587, 40588, 40727, 40729, 40730, 42618, 
            42622, 42623, 42625, 42626, 42729, 42809, 42961, 43389, 43391, 43392, 
            43393, 4434, 4436, 4437, 4470, 4471, 4473, 5, 6, 6872, 6875, 6902, 6903, 
            6905, 7143, 8146, 8298, 8425, 43396, 43397, 43398, 43400, 43401]

# First clean up any existing model files
!rm -f gender_net.caffemodel gender_deploy.prototxt

# Then run:
output_path, gender_stats = download_faces(female_ids, male_ids, delay=1.0)

Progress:
Males: 75/75 (0 IDs remaining)
Females: 69/69 (0 IDs remaining)
Last detection confidence: 1.00



Completed! All IDs have been assigned:
Males: 75/75
Females: 69/69
Images saved in: /Users/kcwaverider/Code/VR_Orgchart/utilities/face_generator/generated_faces


In [3]:
download_faces(female_ids, male_ids, output_dir='generated_faces', delay=1.0)

Progress:
Males: 75/75 (0 IDs remaining)
Females: 69/69 (0 IDs remaining)
Last detection confidence: 1.00



Completed! All IDs have been assigned:
Males: 75/75
Females: 69/69
Images saved in: /Users/kcwaverider/Code/VR_Orgchart/utilities/face_generator/generated_faces


(PosixPath('generated_faces'), {'Male': 75, 'Female': 69})